In [9]:
from pyspark.sql import SparkSession, functions as F

spark = SparkSession.builder.appName("Exploração de Dados - Camada Bronze").getOrCreate()


# Extração

In [10]:
df_yellow = spark.read.table("bronze_db.nyc_taxi_data_yellow")
df_yellow.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+--------------------+------------------+
|vendorid|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|ratecodeid|store_and_fwd_flag|pulocationid|dolocationid|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|  data_hora_ingestao|ano_mes_referencia|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+--------------------+------------------+
|     1.0| 2023-04-01 00:14:49|  2023-04-01 00:45:01|            2.

In [11]:
df_green = spark.read.table("bronze_db.nyc_taxi_data_green")
df_green.show()

+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+--------------------+------------------+
|vendorid|lpep_pickup_datetime|lpep_dropoff_datetime|store_and_fwd_flag|ratecodeid|pulocationid|dolocationid|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|  data_hora_ingestao|ano_mes_referencia|
+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+--------------------+------------------+
|     2.0| 2023-05-01 00:52:10|  2023-05-01

In [17]:
df_yellow_tranformed = df_yellow.selectExpr(
    "vendorid as id_fornecedor",
    "passenger_count as quantidade_passageiros",
    "total_amount as valor_corrida",
    "tpep_pickup_datetime as data_hora_embarque",
    "tpep_dropoff_datetime as data_hora_desembarque",
    "payment_type as id_tipo_pagamento", 
    "ano_mes_referencia",
    '"YELLOW" as tipo_servico',
)

df_green_tranformed = df_green.selectExpr(
    "vendorid as id_fornecedor",
    "passenger_count as quantidade_passageiros",
    "total_amount as valor_corrida",
    "lpep_pickup_datetime as data_hora_embarque",
    "lpep_dropoff_datetime as data_hora_desembarque",
    "payment_type as id_tipo_pagamento", 
    "ano_mes_referencia",
    '"GREEN" as tipo_servico',
)

df = df_yellow_tranformed.union(df_green_tranformed)

df = (
    df.withColumn(
        "nome_fornecedor", 
        F.when(F.col("id_fornecedor") == 1, "Creative Mobile Technologies, LLC")
        .when(F.col("id_fornecedor") == 2, "Curb Mobility, LLC")
        .when(F.col("id_fornecedor") == 6, "Myle Technologies Inc")
        .when(F.col("id_fornecedor") == 7, "Helix")
        .otherwise("FORNECEDOR NÃO IDENTIFICADO")
    )
)

df = (
    df.withColumn(
        "descricao_tipo_pagamento", 
        F.when(F.col("id_tipo_pagamento") == 0, "Viagem com tarifa flexível")
        .when(F.col("id_tipo_pagamento") == 1, "Cartão de crédito")
        .when(F.col("id_tipo_pagamento") == 2, "Dinheiro")
        .when(F.col("id_tipo_pagamento") == 3, "Sem cobrança")
        .when(F.col("id_tipo_pagamento") == 4, "Contestação")
        .when(F.col("id_tipo_pagamento") == 5, "Desconhecido")
        .when(F.col("id_tipo_pagamento") == 6, "Viagem cancelada")
        .otherwise("TIPO DE PAGAMENTO NÃO IDENTIFICADO")
    )
)

df = df.withColumn("indicador_cancelamento", F.when(F.col("id_tipo_pagamento") == 6, 'S').otherwise('N'))
df = df.withColumn("indicador_viagem_sem_cobranca", F.when(F.col("id_tipo_pagamento") == 3, 'S').otherwise('N'))
df = df.withColumn("id", F.expr("uuid()"))

df_viagem_taxi_ny = df.select(
    "id",
    "id_fornecedor",
    "nome_fornecedor",
    "quantidade_passageiros",
    "valor_corrida",
    "data_hora_embarque",
    "data_hora_desembarque",
    "indicador_cancelamento",
    "indicador_viagem_sem_cobranca",
    "id_tipo_pagamento",
    "descricao_tipo_pagamento",
    "tipo_servico",
    "ano_mes_referencia",
)

df_viagem_taxi_ny.show()




+--------------------+-------------+--------------------+----------------------+-------------+-------------------+---------------------+----------------------+-----------------------------+-----------------+------------------------+------------+------------------+
|                  id|id_fornecedor|     nome_fornecedor|quantidade_passageiros|valor_corrida| data_hora_embarque|data_hora_desembarque|indicador_cancelamento|indicador_viagem_sem_cobranca|id_tipo_pagamento|descricao_tipo_pagamento|tipo_servico|ano_mes_referencia|
+--------------------+-------------+--------------------+----------------------+-------------+-------------------+---------------------+----------------------+-----------------------------+-----------------+------------------------+------------+------------------+
|7ecf77e9-9571-48c...|          1.0|Creative Mobile T...|                   0.0|        51.65|2023-05-01 00:33:13|  2023-05-01 00:53:01|                     N|                            N|              1.

In [25]:
df_viagem_taxi_ny.printSchema()

root
 |-- id: string (nullable = false)
 |-- id_fornecedor: double (nullable = true)
 |-- nome_fornecedor: string (nullable = false)
 |-- quantidade_passageiros: double (nullable = true)
 |-- valor_corrida: double (nullable = true)
 |-- data_hora_embarque: timestamp (nullable = true)
 |-- data_hora_desembarque: timestamp (nullable = true)
 |-- indicador_cancelamento: string (nullable = false)
 |-- indicador_viagem_sem_cobranca: string (nullable = false)
 |-- id_tipo_pagamento: double (nullable = true)
 |-- descricao_tipo_pagamento: string (nullable = false)
 |-- tipo_servico: string (nullable = false)
 |-- ano_mes_referencia: string (nullable = true)



In [18]:
"""
    Qual a média de valor total (total_amount) recebido em um mês
    considerando todos os yellow táxis da frota?
"""


_ = (
    df_viagem_taxi_ny
    .filter("tipo_servico = 'YELLOW'") # Somente yellow táxis
    .filter("indicador_cancelamento = 'N'") # Somente corridas não canceladas
    .filter("indicador_viagem_sem_cobranca = 'N'") # Somente corridas com cobrança
    .groupBy("ano_mes_referencia") 
    .agg(F.round(F.avg("valor_corrida"), 2).alias("media_valor_corrida"))
    .orderBy("ano_mes_referencia")
    .show()
)


+------------------+-------------------+
|ano_mes_referencia|media_valor_corrida|
+------------------+-------------------+
|           2023-01|              27.12|
|           2023-02|               27.0|
|           2023-03|              27.91|
|           2023-04|              28.38|
|           2023-05|              29.07|
+------------------+-------------------+



In [24]:
"""
Qual a média de passageiros (passenger_count) por cada hora do dia
que pegaram táxi no mês de maio considerando todos os táxis da
frota?
"""

_ = (
    df_viagem_taxi_ny
    .filter(F.col("ano_mes_referencia") == "2023-05")
    .withColumn("hora_embarque", F.hour(F.col("data_hora_embarque")))
    .withColumn("hora_embarque", F.col("hora_embarque").cast("string"))
    .withColumn("hora_embarque", F.lpad(F.col("hora_embarque"), 2, "0"))
    .groupBy("ano_mes_referencia", "hora_embarque")
    .agg(F.round(F.avg("quantidade_passageiros"), 2).alias("media_passageiros"))
    .orderBy("hora_embarque")
    .show()
)

+------------------+-------------+-----------------+
|ano_mes_referencia|hora_embarque|media_passageiros|
+------------------+-------------+-----------------+
|           2023-05|           00|             1.41|
|           2023-05|           01|             1.42|
|           2023-05|           02|             1.44|
|           2023-05|           03|             1.43|
|           2023-05|           04|             1.39|
|           2023-05|           05|             1.26|
|           2023-05|           06|             1.24|
|           2023-05|           07|             1.25|
|           2023-05|           08|             1.26|
|           2023-05|           09|             1.28|
|           2023-05|           10|             1.32|
|           2023-05|           11|             1.33|
|           2023-05|           12|             1.35|
|           2023-05|           13|             1.35|
|           2023-05|           14|             1.36|
|           2023-05|           15|            